Julia Package `QuantumAlgebra`: https://github.com/jfeist/QuantumAlgebra.jl
```julia
using Pkg
Pkg.add("QuantumAlgebra")
```

Allows to write (somewhat) arbitrary expressions of quantum operators and perform commutator algebra etc with them.

Possible operators (for now): Bosonic ($a$, $a^\dagger$) and two-level systems (Pauli matrices) $\sigma_{x,y,z}$.

Documentation at https://jfeist.github.io/QuantumAlgebra.jl/stable/

In [1]:
using QuantumAlgebra

In [2]:
adag()*a()

QuantumAlgebra.OpProd(adag{Tuple{}}(()), a{Tuple{}}(()))

All expressions are reduced to a "canonical" form (normal ordering + some extra rules)

In [3]:
a()*adag()

QuantumAlgebra.OpSum(scal{Int64}(1), QuantumAlgebra.OpProd(adag{Tuple{}}(()), a{Tuple{}}(())))

In [4]:
σx()*σy()

QuantumAlgebra.OpProd(scal{Complex{Rational{Int64}}}(0//1 + 1//1*im), QuantumAlgebra.σ{Tuple{}}(QuantumAlgebra.z, ()))

In [5]:
σy()*σz()

QuantumAlgebra.OpProd(scal{Complex{Rational{Int64}}}(0//1 + 1//1*im), QuantumAlgebra.σ{Tuple{}}(QuantumAlgebra.x, ()))

In [6]:
σy()*σy()

scal{Int64}(1)

Operators can have indices.

In [7]:
a(:k)*adag(:k)

QuantumAlgebra.OpSum(scal{Int64}(1), QuantumAlgebra.OpProd(adag{Tuple{Symbol}}((:k,)), a{Tuple{Symbol}}((:k,))))

Symbolic indices can be equal

In [8]:
a(:k)*adag(:j)

QuantumAlgebra.OpSum(QuantumAlgebra.δ((:j, :k)), QuantumAlgebra.OpProd(adag{Tuple{Symbol}}((:j,)), a{Tuple{Symbol}}((:k,))))

**Sums**: Can represent analytic sums over indices

In [9]:
s = ∑(:i,adag(:i))

OpSumAnalytic(:i, adag{Tuple{Symbol}}((:i,)))

In [10]:
s*a(:j)

OpSumAnalytic(:i, QuantumAlgebra.OpProd(adag{Tuple{Symbol}}((:i,)), a{Tuple{Symbol}}((:j,))))

Sum indices are assumed to run over **all** possible values.  
Summing over $\delta_{i,j}$ removes the sum.

In [11]:
a(:j)*s

QuantumAlgebra.OpSum(scal{Int64}(1), OpSumAnalytic(:i, QuantumAlgebra.OpProd(adag{Tuple{Symbol}}((:i,)), a{Tuple{Symbol}}((:j,)))))

In [12]:
a(:k)*s

QuantumAlgebra.OpSum(scal{Int64}(1), OpSumAnalytic(:i, QuantumAlgebra.OpProd(adag{Tuple{Symbol}}((:i,)), a{Tuple{Symbol}}((:k,)))))

In [13]:
a(:k)*∑(:i,adag(:i)*adag(:i))

QuantumAlgebra.OpSum(QuantumAlgebra.OpProd(scal{Int64}(2), adag{Tuple{Symbol}}((:k,))), OpSumAnalytic(:i, QuantumAlgebra.OpProd(adag{Tuple{Symbol}}((:i,)), QuantumAlgebra.OpProd(adag{Tuple{Symbol}}((:i,)), a{Tuple{Symbol}}((:k,))))))

**Multiple indices, nested sums, etc...**

In [14]:
a(:i,:j,:μ) * adag(:m,:n,:μ)

QuantumAlgebra.OpSum(QuantumAlgebra.OpProd(QuantumAlgebra.δ((:i, :m)), QuantumAlgebra.δ((:j, :n))), QuantumAlgebra.OpProd(adag{Tuple{Symbol,Symbol,Symbol}}((:m, :n, :μ)), a{Tuple{Symbol,Symbol,Symbol}}((:i, :j, :μ))))

In [15]:
∑(:j, ∑(:i,a(:i,:j,:μ))) * adag(:m,:n,:μ)

QuantumAlgebra.OpSum(scal{Int64}(1), OpSumAnalytic(:i, OpSumAnalytic(:j, QuantumAlgebra.OpProd(adag{Tuple{Symbol,Symbol,Symbol}}((:m, :n, :μ)), a{Tuple{Symbol,Symbol,Symbol}}((:i, :j, :μ))))))

In [16]:
∑(:i,a(:i)) * ∑(:k,adag(:k))

QuantumAlgebra.OpSum(OpSumAnalytic(:i, scal{Int64}(1)), OpSumAnalytic(:i, OpSumAnalytic(:k, QuantumAlgebra.OpProd(adag{Tuple{Symbol}}((:k,)), a{Tuple{Symbol}}((:i,))))))

Caution: implementation of analytic sums not fully finished (especially for nested sums).

**Parameters**

In [17]:
s = ∑(:j,param(:ω,:j)*adag(:j))

OpSumAnalytic(:j, QuantumAlgebra.OpProd(param{Tuple{Symbol}}(:ω, 'n', (:j,)), adag{Tuple{Symbol}}((:j,))))

string-based input form:

In [18]:
s = ∑(:j,Pr"ω_j"*adag(:j))

OpSumAnalytic(:j, QuantumAlgebra.OpProd(param{Tuple{Symbol}}(:ω, 'r', (:j,)), adag{Tuple{Symbol}}((:j,))))

In [19]:
a(:i)*s

QuantumAlgebra.OpSum(param{Tuple{Symbol}}(:ω, 'r', (:i,)), OpSumAnalytic(:j, QuantumAlgebra.OpProd(param{Tuple{Symbol}}(:ω, 'r', (:j,)), QuantumAlgebra.OpProd(adag{Tuple{Symbol}}((:j,)), a{Tuple{Symbol}}((:i,))))))

Back to the original motivation:
**Emitter + many-mode Hamiltonian**

In [20]:
H = 1//2*Pr"Ω"*σz() + Pr"E(t)"*σx() + ∑(:i,Pr"ω_i"*adag(:i)*a(:i)) + ∑(:i,Pr"g_i"*(adag(:i)+a(:i))*σx())

QuantumAlgebra.OpSum(QuantumAlgebra.OpProd(param{Tuple{}}(Symbol("E(t)"), 'r', ()), QuantumAlgebra.σ{Tuple{}}(QuantumAlgebra.x, ())), QuantumAlgebra.OpSum(QuantumAlgebra.OpProd(scal{Rational{Int64}}(1//2), QuantumAlgebra.OpProd(param{Tuple{}}(:Ω, 'r', ()), QuantumAlgebra.σ{Tuple{}}(QuantumAlgebra.z, ()))), QuantumAlgebra.OpSum(OpSumAnalytic(:i, QuantumAlgebra.OpProd(param{Tuple{Symbol}}(:ω, 'r', (:i,)), QuantumAlgebra.OpProd(adag{Tuple{Symbol}}((:i,)), a{Tuple{Symbol}}((:i,))))), QuantumAlgebra.OpSum(OpSumAnalytic(:i, QuantumAlgebra.OpProd(param{Tuple{Symbol}}(:g, 'r', (:i,)), QuantumAlgebra.OpProd(adag{Tuple{Symbol}}((:i,)), QuantumAlgebra.σ{Tuple{}}(QuantumAlgebra.x, ())))), OpSumAnalytic(:i, QuantumAlgebra.OpProd(param{Tuple{Symbol}}(:g, 'r', (:i,)), QuantumAlgebra.OpProd(a{Tuple{Symbol}}((:i,)), QuantumAlgebra.σ{Tuple{}}(QuantumAlgebra.x, ()))))))))

In [21]:
comm(σz(),H)

QuantumAlgebra.OpSum(QuantumAlgebra.OpProd(scal{Complex{Int64}}(0 + 2im), QuantumAlgebra.OpProd(param{Tuple{}}(Symbol("E(t)"), 'r', ()), QuantumAlgebra.σ{Tuple{}}(QuantumAlgebra.y, ()))), QuantumAlgebra.OpSum(QuantumAlgebra.OpProd(scal{Complex{Int64}}(0 + 2im), OpSumAnalytic(:i, QuantumAlgebra.OpProd(param{Tuple{Symbol}}(:g, 'r', (:i,)), QuantumAlgebra.OpProd(adag{Tuple{Symbol}}((:i,)), QuantumAlgebra.σ{Tuple{}}(QuantumAlgebra.y, ()))))), QuantumAlgebra.OpProd(scal{Complex{Int64}}(0 + 2im), OpSumAnalytic(:i, QuantumAlgebra.OpProd(param{Tuple{Symbol}}(:g, 'r', (:i,)), QuantumAlgebra.OpProd(a{Tuple{Symbol}}((:i,)), QuantumAlgebra.σ{Tuple{}}(QuantumAlgebra.y, ())))))))

In [22]:
comm(adag(:j)*σz(),H)

QuantumAlgebra.OpSum(QuantumAlgebra.OpProd(scal{Complex{Rational{Int64}}}(0//1 + 1//1*im), QuantumAlgebra.OpProd(param{Tuple{Symbol}}(:g, 'r', (:j,)), QuantumAlgebra.σ{Tuple{}}(QuantumAlgebra.y, ()))), QuantumAlgebra.OpSum(QuantumAlgebra.OpProd(scal{Complex{Int64}}(0 + 2im), QuantumAlgebra.OpProd(param{Tuple{}}(Symbol("E(t)"), 'r', ()), QuantumAlgebra.OpProd(adag{Tuple{Symbol}}((:j,)), QuantumAlgebra.σ{Tuple{}}(QuantumAlgebra.y, ())))), QuantumAlgebra.OpSum(QuantumAlgebra.OpProd(scal{Int64}(-1), QuantumAlgebra.OpProd(param{Tuple{Symbol}}(:ω, 'r', (:j,)), QuantumAlgebra.OpProd(adag{Tuple{Symbol}}((:j,)), QuantumAlgebra.σ{Tuple{}}(QuantumAlgebra.z, ())))), QuantumAlgebra.OpSum(QuantumAlgebra.OpProd(scal{Complex{Int64}}(0 + 2im), OpSumAnalytic(:i, QuantumAlgebra.OpProd(param{Tuple{Symbol}}(:g, 'r', (:i,)), QuantumAlgebra.OpProd(adag{Tuple{Symbol}}((:i,)), QuantumAlgebra.OpProd(adag{Tuple{Symbol}}((:j,)), QuantumAlgebra.σ{Tuple{}}(QuantumAlgebra.y, ())))))), QuantumAlgebra.OpProd(scal{Comp